In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:13 https://developer.

In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").getOrCreate()

In [4]:
# Read in dataset
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

# Show dataframe
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [5]:
# Create the vine_table DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R3EQSTM9PWRAL|          3|            0|          0|   N|                Y|
| RBWPRK17XKIXD|          5|            0|          0|   N|                Y|
| RRSLOAF273XFC|          5|            1|          2|   N|                Y|
|R3S8W9Q6SWIT8O|          4|            0|          0|   N|                Y|
|R3QQ6NSLRVBFJC|          4|            0|          0|   N|                Y|
|R2XYDBMHUVJCSX|          2|            0|          1|   N|                Y|
|R1L94ESVC657A9|          5|            0|          0|   N|                Y|
|R3DI4B8LDWFQ3K|          1|            3|          4|   N|                Y|
|R1G7VV4HCXUQQL|          5|            0|          0|   N|                Y|
|R3TTNYN1TAMUSU|          5|            0|          0|   N|     

In [7]:
# Filter to show rows where the total_votes count is equal to or greater than 20
vine_20_df = vine_df.filter("total_votes>=20")
vine_20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
|R1M8EEGH2TVS0Q|          1|            6|         29|   N|                Y|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|     

In [8]:
# Filter to show Helpful Votes divided by Total Votes >= 50%
vine_20_50_df = vine_20_df.filter("helpful_votes / total_votes>=0.50")
vine_20_50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|                Y|
| RYA9ESIATW4EC|          1|           23|         27|   N|     

In [20]:
# Filter to show Paid reviews, i.e. Vine = Yes & show statistics
vine_20_50_Y_df = vine_20_50_df.filter(vine_20_50_df['Vine'] == 'Y')
vine_20_50_Y_df.describe().show()

+-------+--------------+------------------+------------------+------------------+----+-----------------+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+--------------+------------------+------------------+------------------+----+-----------------+
|  count|           497|               497|               497|               497| 497|              497|
|   mean|          null| 3.943661971830986| 79.88732394366197| 89.29376257545272|null|             null|
| stddev|          null|1.2135180588087573|151.51957242183875|163.20551010785294|null|             null|
|    min|R108U30Q0YXJYO|                 1|                11|                20|   Y|                N|
|    max| RZK960YBXT2TC|                 5|              1843|              2042|   Y|                Y|
+-------+--------------+------------------+------------------+------------------+----+-----------------+



In [21]:
# Filter to show Paid and 5-star reviews, i.e. Vine = Yes & show statistics
vine_20_50_Y_5_stars_df = vine_20_50_Y_df.filter(vine_20_50_Y_df['star_rating'] == 5)
vine_20_50_Y_5_stars_df.describe().show()

+-------+--------------+-----------+-----------------+------------------+----+-----------------+
|summary|     review_id|star_rating|    helpful_votes|       total_votes|vine|verified_purchase|
+-------+--------------+-----------+-----------------+------------------+----+-----------------+
|  count|           220|        220|              220|               220| 220|              220|
|   mean|          null|        5.0|64.26818181818182| 71.67727272727272|null|             null|
| stddev|          null|        0.0|102.8768199298117|107.88278110190451|null|             null|
|    min|R108U30Q0YXJYO|          5|               13|                20|   Y|                N|
|    max| RYX1ZN8R6OOPK|          5|             1088|              1127|   Y|                Y|
+-------+--------------+-----------+-----------------+------------------+----+-----------------+



In [22]:
# Filter to show Unpaid reviews, i.e. Vine = No & show statistics
vine_20_50_N_df = vine_20_50_df.filter(vine_20_50_df['Vine'] == 'N')
vine_20_50_N_df.describe().show()

+-------+--------------+------------------+------------------+-----------------+------+-----------------+
|summary|     review_id|       star_rating|     helpful_votes|      total_votes|  vine|verified_purchase|
+-------+--------------+------------------+------------------+-----------------+------+-----------------+
|  count|        120863|            120863|            120863|           120863|120863|           120863|
|   mean|          null| 3.945682301448748|52.797415255289046|60.17075531800468|  null|             null|
| stddev|          null|1.5568640132223812|135.62029779658604|143.1546262213917|  null|             null|
|    min|R10002WDZH1F0U|                 1|                10|               20|     N|                N|
|    max| RZZXTGFR2H84U|                 5|             16401|            17182|     N|                Y|
+-------+--------------+------------------+------------------+-----------------+------+-----------------+



In [23]:
# Filter to show Unpaid reviews and 5-star reviews, i.e. Vine = No & show statistics
vine_20_50_N_5_stars_df = vine_20_50_N_df.filter(vine_20_50_N_df['star_rating'] == 5)
vine_20_50_N_5_stars_df.describe().show()

+-------+--------------+-----------+------------------+------------------+-----+-----------------+
|summary|     review_id|star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+--------------+-----------+------------------+------------------+-----+-----------------+
|  count|         74470|      74470|             74470|             74470|74470|            74470|
|   mean|          null|        5.0| 54.16539546125957| 60.58868000537129| null|             null|
| stddev|          null|        0.0|144.09505283488616|151.19025363343593| null|             null|
|    min|R10002WDZH1F0U|          5|                10|                20|    N|                N|
|    max| RZZXTGFR2H84U|          5|             16401|             17182|    N|                Y|
+-------+--------------+-----------+------------------+------------------+-----+-----------------+

